In [50]:
import pandas as pd
import requests
import os
import statistics as stat

We import a dataset from kaggle, with the list of the top 5000 board games on the website BoardGameGeek, ranked by user rating. The data was extracted on February 2019.  
First we read the dataset from a csv and store it in a pandas dataframe.

In [5]:
df = pd.read_csv("input/topBGs2019")
df.head()

,rank,bgg_url,game_id,names,min_players,max_players,avg_time,min_time,max_time,year,...,age,mechanic,owned,category,designer,publisher,weight,expands,reimplements,num_fans
0,1,https://boardgamegeek.com/boardgame/174430/glo...,174430,Gloomhaven,1,4,120,60,120,2017,...,12,"Campaign / Battle Card Driven, Cooperative Pla...",36778,"Adventure, Exploration, Fantasy, Fighting, Min...",Isaac Childres,Cephalofair Games,3.7768,NaN,NaN,4413
1,2,https://boardgamegeek.com/boardgame/161936/pan...,161936,Pandemic Legacy: Season 1,2,4,60,60,60,2015,...,13,"Action Point Allowance System, Cooperative Pla...",47498,"Environmental, Medical","Rob Daviau, Matt Leacock","Z-Man Games, Inc.",2.8323,NaN,30549.0,2168
2,3,https://boardgamegeek.com/boardgame/182028/thr...,182028,Through the Ages: A New Story of Civilization,2,4,120,120,120,2015,...,14,"Action Point Allowance System, Auction/Bidding...",18489,"Card Game, Civilization, Economic",Vlaada Chvátil,Czech Games Edition,4.3715,NaN,25613.0,1054
3,4,https://boardgamegeek.com/boardgame/167791/ter...,167791,Terraforming Mars,1,5,120,120,120,2016,...,12,"Card Drafting, Hand Management, Set Collection...",45204,"Economic, Environmental, Industry / Manufactur...",Jacob Fryxelius,FryxGames,3.2344,NaN,NaN,2718
4,5,https://boardgamegeek.com/boardgame/12333/twil...,12333,Twilight Struggle,2,2,180,120,180,2005,...,13,"Area Control / Area Influence, Campaign / Batt...",45955,"Modern Warfare, Political, Wargame","Ananda Gupta, Jason Matthews",GMT Games,3.5627,NaN,NaN,3639


In [3]:
df["names"][0]float(prices[0])

'Gloomhaven'

In [23]:
# We set the client_id to be this one for now, which is public.
client_id = "qaLdCno66F"

In [70]:
def requestAtlas(endpoint):
       
    baseUrl = "https://www.boardgameatlas.com/api/"
    url = baseUrl+endpoint
    print(f"Requesting data from {url}")
    
    res = requests.get(url)
    if res.status_code != 200:
        print(res.text)
        raise ValueError("Bad Response")
    return res.json()

#requestAtlas("search?name=Catan&client_id=SB1VGnDv7M")
#gloomhaven = requestAtlas("search?name=Gloomhaven&client_id=SB1VGnDv7M&exact=True&order_by=popularity&limit=1")

gloomhaven['games'][0]['id'] # This is the code to get the boardgameAtlas id that we need to get price information

'RLlDWHh7hR'

In [39]:
# Searching prices by ID
# this is the format for the search by price: game/prices?game_id=6FmFeux5xH&client_id=qaLdCno66F

gloom_id = gloomhaven['games'][0]['id']
# gloomPrice = requestAtlas(f'game/prices?game_id={gloom_id}&client_id={client_id}')
# print(gloomPrice)
gloomPrice['prices']

gloomPrice['prices'][5:7]

[{'price': '0.00',
  'price_text': 'Out of Stock',
  'url': 'https://www.boardgameatlas.com/api/price?client_id=W0AQGbjlZE&store=game_nerdz&id=Dpy9NjQf4V',
  'store_name': 'Game Nerdz',
  'updated_at': '2020-01-31T13:30:42.419Z',
  'country': 'US'},
 {'price': '109.99',
  'price_text': '$109.99',
  'url': 'https://www.boardgameatlas.com/api/price?client_id=W0AQGbjlZE&store=card_haus&id=ovBuh4b83r',
  'store_name': 'Card Haus',
  'updated_at': '2020-01-31T13:30:39.289Z',
  'country': 'US'}]

In [56]:
prices = []
for price in gloomPrice['prices']:
    try:
        if float(price['price']) > 0:
            prices.append(float(price['price']))
    except:
        continue

avgPrice = stat.mean(prices)
avgPrice

102.84

## Combining into a function

Now that we have prototyped our pipeline, let us make a function that gets us a game-id, and another that gets us the average price for a game when give the id.

In [77]:
def getGameID (gamename, client_id="qaLdCno66F", exact=False, limit=1):
    # When give a gamename and a client_id, this function returns the game_id used in the API of boardgameatlas. 
    # The function returns the most popular game when searching by 
    game = requestAtlas(f'search?name={gamename}&client_id={client_id}&order_by=popularity&limit={limit}&exact={exact}')
    return game['games'][0]['id']

#id = getGameID(df.names[5]) #Works great!

def getAvgPrice(gameID, client_id="qaLdCno66F"):
    prices = requestAtlas(f'game/prices?game_id={gameID}&client_id={client_id}')
    
    pfloats = []
    for p in prices['prices']:
        try:
            if float(p['price']) > 0:
                pfloats.append(float(p['price']))
                print(float(p['price']))
        except:
            continue
    return stat.mean(pfloats)



Requesting data from https://www.boardgameatlas.com/api/game/prices?game_id=Qq2rPNOpoO&client_id=qaLdCno66F
89.95
83.98


86.965